In [1]:
# Thay thế tất cả các cell !pip install bằng cell duy nhất này
!pip install llama-index-graph-stores-neo4j llama-index-vector-stores-chroma llama-index-embeddings-huggingface llama-index-llms-openrouter python-dotenv sentence-transformers

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import os
import chromadb
from dotenv import load_dotenv
import logging

from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter 

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
load_dotenv()


Settings.llm = OpenRouter(
    api_key=os.getenv("OPENAI_API_KEY"), 
    model="mistralai/mistral-7b-instruct:free",
    default_headers={
        "HTTP-Referer": os.getenv("YOUR_SITE_URL", "http://localhost:8888"), 
        "X-Title": os.getenv("YOUR_SITE_NAME", "VectorGraphRAG PoC"),     
    },
    max_tokens=512,
    temperature=0.1,
)
logging.info(f"✅ LLM đã được cấu hình để sử dụng model '{Settings.llm.model}' từ OpenRouter.")


# Cấu hình Embedding Model (giữ nguyên)
Settings.embed_model = HuggingFaceEmbedding(model_name="bkai-foundation-models/vietnamese-bi-encoder")

# --- 2. DỮ LIỆU MẪU (giữ nguyên) ---
text_docs = [
    Document(
        text="Triết lý thiết kế của Steve Jobs tại Apple luôn tập trung vào sự đơn giản, trực quan và đặt trải nghiệm người dùng làm trung tâm. Ông tin rằng sản phẩm không chỉ cần mạnh mẽ về chức năng mà còn phải là một tác phẩm nghệ thuật, đẹp từ trong ra ngoài. Điều này đã định hình nên các sản phẩm biểu tượng như iPhone và MacBook.",
        metadata={"source": "Phân tích nội bộ"}
    ),
    Document(
        text="iPhone 15, ra mắt năm 2023, tiếp tục kế thừa di sản thiết kế của Apple với khung viền titan, cổng USB-C và hệ thống camera cải tiến mạnh mẽ. Nó được xem là một trong những smartphone hàng đầu thị trường, thể hiện rõ triết lý của công ty.",
        metadata={"source": "Đánh giá sản phẩm TechReview"}
    ),
    Document(
        text="Cuộc đối đầu giữa Apple và Samsung là một trong những câu chuyện kinh điển của làng công nghệ. Trong khi Apple tập trung vào hệ sinh thái khép kín và sự tối ưu hóa phần cứng-phần mềm, Samsung lại đa dạng hóa sản phẩm với nhiều phân khúc và tiên phong trong các công nghệ màn hình gập.",
        metadata={"source": "Bài báo kinh doanh"}
    ),
]
graph_triplets = [
    ("Steve Jobs", "FOUNDED", "Apple"),
    ("Apple", "PRODUCES", "iPhone 15"),
    ("Apple", "HAS_PHILOSOPHY", "Thiết kế tối giản và trải nghiệm người dùng"),
    ("Samsung", "PRODUCES", "Galaxy S24"),
    ("Samsung", "COMPETES_WITH", "Apple")
]

# --- 3. INGESTION VÀO NEO4J (GraphDB) (giữ nguyên) ---
logging.info("--- Bắt đầu Ingest vào Neo4j ---")
graph_store = Neo4jGraphStore(
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    url=os.getenv("NEO4J_URI"),
    database="neo4j",
    refresh_schema=False
)
graph_store.query("MATCH (n) DETACH DELETE n")
for subj, pred, obj in graph_triplets:
    graph_store.upsert_triplet(subj, pred, obj)
logging.info("--- Ingest vào Neo4j Hoàn tất ---")


# --- 4. INGESTION VÀO CHROMADB (VectorDB) (giữ nguyên) ---
logging.info("--- Bắt đầu Ingest vào ChromaDB ---")
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("vector_graph_rag_poc")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
vector_index = VectorStoreIndex.from_documents(
    text_docs, vector_store=vector_store
)
logging.info("--- Ingest vào ChromaDB Hoàn tất ---")

2025-06-30 10:33:26,738 - INFO - ✅ LLM đã được cấu hình để sử dụng model 'mistralai/mistral-7b-instruct:free' từ OpenRouter.
2025-06-30 10:33:26,741 - INFO - Load pretrained SentenceTransformer: bkai-foundation-models/vietnamese-bi-encoder
2025-06-30 10:33:30,988 - INFO - 2 prompts are loaded, with the keys: ['query', 'text']
2025-06-30 10:33:30,989 - INFO - --- Bắt đầu Ingest vào Neo4j ---
2025-06-30 10:33:31,031 - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_1ed05907 FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: '\n                CREATE CONSTRAINT IF NOT EXISTS FOR (n:Entity) REQUIRE n.id IS UNIQUE;\n                '
2025-06-30 10:33:31,141 - INFO - --- Ingest vào Neo4j Hoàn tất ---
2025-06-30 10:3

In [13]:
import json
from llama_index.core.llms import ChatMessage

def transform_query(query: str) -> dict:
    """
    Sử dụng LLM để phân rã câu hỏi phức tạp.
    """
    prompt = f"""
Bạn là một trợ lý AI hữu ích, chuyên phân tích và phân rã các câu hỏi phức tạp của người dùng thành các nhiệm vụ con cho một hệ thống RAG. Mục tiêu của bạn là tách câu hỏi gốc thành một danh sách các câu hỏi con độc lập và trích xuất các thực thể chính. Một số câu hỏi con sẽ được dùng để truy vấn cơ sở dữ liệu vector (để lấy thông tin mô tả), và các thực thể sẽ được dùng để truy vấn đồ thị tri thức (để lấy dữ liệu thực tế và các mối quan hệ).

Dựa trên câu hỏi của người dùng, hãy tạo một đối tượng JSON chứa hai danh sách: 'vector_search_queries' và 'graph_entities'.

- 'vector_search_queries': Chứa các câu hỏi mô tả, đầy đủ ý nghĩa.
- 'graph_entities': Chứa các thực thể chính được xác định trong câu hỏi.

User Question: "{query}"

{{
  "vector_search_queries": [],
  "graph_entities": []
}}
"""
    response = Settings.llm.chat([ChatMessage(role="user", content=prompt)])
    try:
        # Cố gắng trích xuất JSON nếu nó nằm trong khối markdown
        json_match = re.search(r"```json\n(.*?)\n```", content, re.S)
        if json_match:
            content = json_match.group(1)

        result = json.loads(content)
        return result
    except (json.JSONDecodeError, TypeError) as e:
        # Fallback nếu LLM không trả về JSON hợp lệ
        logging.error(f"Lỗi khi phân tích JSON từ LLM: {e}")
        logging.error(f"Nội dung nhận được từ LLM: '{content}'")
        logging.info("Sử dụng fallback: trả về câu hỏi gốc làm truy vấn vector.")
        return {"vector_search_queries": [query], "graph_entities": []}

In [14]:
import asyncio
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.schema import NodeWithScore, QueryBundle
from typing import List, Optional
import re

# Lấy lại graph_store đã khởi tạo ở bước trước
graph_store = Neo4jGraphStore(
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    url=os.getenv("NEO4J_URI"),
    database="neo4j",
    refresh_schema=False
)

def get_connected_entities(entity: str) -> List[str]:
    """Truy vấn Neo4j để tìm các thực thể có kết nối trực tiếp."""
    try:
        query = f"""
        MATCH (e1 {{id: "{entity}"}})-- (e2)
        RETURN e2.id
        """
        result = graph_store.query(query)
        return [item['e2.id'] for item in result] if result else []
    except:
        return []

class GraphSignalBooster(BaseNodePostprocessor):
    """
    Tăng điểm cho các node dựa trên tín hiệu từ đồ thị tri thức.
    """
    # BƯỚC 1: Khai báo các thuộc tính của lớp trước
    graph_entities: List[str] = []
    weight: float = 1.5
    
    # BƯỚC 2: Khai báo một thuộc tính riêng tư để lưu các thực thể liên quan
    # Chúng ta dùng _related_entities để tránh xung đột với Pydantic
    _related_entities: set = set()

    # BƯỚC 3: Sửa lại hàm __init__
    def __init__(
        self,
        graph_entities: List[str],
        weight: float = 1.5,
        **kwargs
    ):
        # Gọi super().__init__ và truyền các giá trị vào
        # Pydantic sẽ tự động gán self.graph_entities và self.weight
        super().__init__(graph_entities=graph_entities, weight=weight, **kwargs)
        
        # Bây giờ, chúng ta có thể thực hiện logic tính toán
        self._related_entities = set(self.graph_entities)
        for entity in self.graph_entities:
            connected = get_connected_entities(entity)
            self._related_entities.update(connected)
        print(f"[GraphSignalBooster] Các thực thể liên quan từ đồ thị: {self._related_entities}")

    def _postprocess_nodes(
        self, nodes: List[NodeWithScore], query_bundle: Optional[QueryBundle] = None
    ) -> List[NodeWithScore]:
        # BƯỚC 4: Sử dụng _related_entities thay vì self.related_entities
        boosted_nodes = []
        for node_with_score in nodes:
            node_text = node_with_score.node.get_content().lower()
            boost_factor = 1.0
            
            found_entities = []
            for entity in self._related_entities: # <-- SỬA Ở ĐÂY
                if re.search(r'\\b' + re.escape(entity.lower()) + r'\\b', node_text):
                    # Sử dụng self.weight đã được Pydantic gán
                    boost_factor = self.weight
                    found_entities.append(entity)
            
            if boost_factor > 1.0:
                print(f"[GraphSignalBooster] Tăng điểm cho node (source: {node_with_score.node.metadata.get('source')}) vì chứa các thực thể liên quan: {found_entities}")

            node_with_score.score *= boost_factor
            boosted_nodes.append(node_with_score)
        
        boosted_nodes.sort(key=lambda x: x.score, reverse=True)
        return boosted_nodes

2025-06-30 10:38:56,430 - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_1ed05907 FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: '\n                CREATE CONSTRAINT IF NOT EXISTS FOR (n:Entity) REQUIRE n.id IS UNIQUE;\n                '


In [8]:
import asyncio
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
# from llama_index.postprocessor.sentence_transformer_rerank import SentenceTransformerRerank
from llama_index.core.postprocessor import SentenceTransformerRerank

async def retrieve_from_vector_db(retriver, queries: List[str]) -> List[NodeWithScore]:
    tasks = [retriver.aretrieve(q) for q in queries]
    nested_results = await asyncio.gather(*tasks)
    
    all_nodes = [node for result in nested_results for node in result]
    
    for query, result in zip(queries, nested_results):
        print(f" Kết quả từ VectorDB cho {query}, được tìm thấy trong {len(result)} tài liệu")
        
    return all_nodes 

async def retrieve_from_graph_db(graph_store,entities: List[str]) -> List[Document]:
    async def query_entity(entity):
        cypher_query = f"MATCH (n {{id:'{entity}'}})-[r]-(m) RETURN n.id"
        try:
            graph_results = await asyncio.to_thread(graph_store.query,cypher_query)
            
            if graph_results:
                print(f"  > Kết quả từ GraphDB cho '{entity}': Đã tìm thấy {len(graph_results)} mối quan hệ.")
                text_result = f"Thông tin từ đồ thị về '{entity}':\n" + "\n".join([f"- {res['n.id']} {res['type(r)']} {res['m.id']}" for res in graph_results])
                return Document(text=text_result, metadata={"source": "GraphDB"})
            return None
        except Exception as e:
            print(f"  > Lỗi khi truy vấn GraphDB cho '{entity}': {e}")
            return None

    tasks = [query_entity(entity) for entity in entities]
    results = await asyncio.gather(*tasks)
    # Lọc ra những kết quả không phải None
    return [doc for doc in results if doc is not None]

async def run_hybrid_rag_pipeline(user_query:str):
    
    # Query Transform
    transformed = transform_query(user_query)
    vector_queries = transformed.get('vector_search_queries',[user_query])
    graph_entities = transformed.get('graph_entities',[])
    
    print(f"Vector Queries: {vector_queries}")
    print(f"Graph Entities: {graph_entities}")
    
    # Paralle Retrieval
    vector_retriever = VectorIndexRetriever(index=vector_index,similarity_top_k=5)
    
    vector_task = retrieve_from_vector_db(vector_retriever,vector_queries)
    graph_task = retrieve_from_graph_db(graph_store,graph_entities)
    
    all_retrieved_nodes, graph_retrieved_text = await asyncio.gather(vector_task,graph_task)
    
    # Combine Results
    unique_nodes = {node.get_content(): node for node in all_retrieved_nodes}
    combined_nodes = list(unique_nodes.values())
    
    for doc in graph_retrieved_text:
        combined_nodes.append(doc)
        
    # Reranking 
    reranker = SentenceTransformerRerank(model="BAAI/bge-reranker-v2-m3",top_n=5)
    graph_booster = GraphSignalBooster(graph_entities=graph_entities)
    
    response_synthesizer = get_response_synthesizer(
        response_mode="compact"
    )
    
    dummy_retriever = VectorIndexRetriever(index=vector_index,similarity_top_k=1)
    
    query_engine = RetrieverQueryEngine.from_args(
        retriever=dummy_retriever,
        response_synthesizer=response_synthesizer,
        node_postprocessor=[reranker,graph_booster]
    )
    
    if hasattr(query_engine, 'asynthesize'):
        final_response = await query_engine.asynthesize(
            query_bundle=QueryBundle(user_query),
            nodes=combined_nodes,
        )
    else: # Fallback cho phiên bản cũ hơn
         final_response = query_engine.synthesize(
            query_bundle=QueryBundle(user_query),
            nodes=combined_nodes,
        )
         
    print(f"Final Response: {final_response}")
    
    
    print("\n--- Các nguồn đã sử dụng để tạo câu trả lời: ---")
    for node in final_response.source_nodes:
        print(f"- Điểm: {node.score:.4f} | Nguồn: {node.metadata.get('source', 'N/A')}")
        print(f"  Nội dung: {node.get_content().strip()[:150]}...")
    
    




In [16]:
# complex_query = "Phân tích triết lý thiết kế của Steve Jobs và nó ảnh hưởng đến iPhone 15 như thế nào, một sản phẩm cạnh tranh với Samsung?"
# await run_hybrid_rag_pipeline(complex_query)